# Vision Language Models com DSPy

Trabalhar com imagens em DSPy é idêntico a trabalhar com texto: declare a Signature, o framework cuida do resto.


## Setup

Configuração inicial: modelo VLM e API key.


In [ ]:
import dspy
import os

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY', 'your-api-key-here')
vision_model = dspy.LM("gemini/gemini-2.5-flash", api_key=GEMINI_API_KEY)
dspy.configure(lm=vision_model)


## Carregando Imagens

`dspy.Image` suporta: `from_url()`, `from_path()`, `from_bytes()`, `from_pil()`. A mesma Signature funciona para qualquer origem.


In [ ]:
image_url = "https://raw.githubusercontent.com/AdoHaha/dspy_fun/refs/heads/main/example_files/image_numbers.png"
numbers_image = dspy.Image.from_url(image_url)

from IPython.display import Image, display
display(Image(image_url, width=400))


## Signatures com Imagens

Use `dspy.Image` como tipo de campo. Combina com campos textuais. Outputs estruturados funcionam automaticamente.


In [ ]:
from typing import List, Dict

class ImageDescription(dspy.Signature):
    """Descreve o conteúdo de uma imagem de forma detalhada."""
    image: dspy.Image = dspy.InputField(desc="Imagem para análise")
    description: str = dspy.OutputField(desc="Descrição detalhada do conteúdo da imagem")

image_describer = dspy.Predict(ImageDescription)
result = image_describer(image=numbers_image)
result.description


## Detecção com Bounding Boxes

Outputs estruturados (List[Dict]) retornam JSON automaticamente. Coordenadas normalizadas (0-1000).


In [ ]:
class NumberDetections(dspy.Signature):
    """Detecta todos os números (não dígitos isolados) na imagem e retorna bounding boxes.
    
    Coordenadas normalizadas (0-1000) em formato xyxy: x_min, y_min, x_max, y_max.
    Retorna lista vazia se nenhum número for encontrado.
    """
    image: dspy.Image = dspy.InputField(desc="Imagem para análise")
    boxes: List[Dict] = dspy.OutputField(
        desc="Um dict por número com coordenadas normalizadas (0-1000): "
             "{'x_min': int, 'y_min': int, 'x_max': int, 'y_max': int, 'number': float}"
    )

number_detector = dspy.Predict(NumberDetections)
detections = number_detector(image=numbers_image)
detections.boxes


## Visualização

Função auxiliar para desenhar bounding boxes sobre a imagem.


In [ ]:
import urllib.request
from PIL import Image, ImageDraw
from io import BytesIO

def draw_detections(image_url: str, detections: List[Dict]) -> Image.Image:
    """Desenha bounding boxes e labels sobre a imagem."""
    with urllib.request.urlopen(image_url) as response:
        img_data = response.read()
    img = Image.open(BytesIO(img_data))
    
    draw = ImageDraw.Draw(img)
    for detection in detections:
        x_min = int(detection["x_min"] * img.width / 1000)
        y_min = int(detection["y_min"] * img.height / 1000)
        x_max = int(detection["x_max"] * img.width / 1000)
        y_max = int(detection["y_max"] * img.height / 1000)
        
        draw.rectangle([(x_min, y_min), (x_max, y_max)], outline="red", width=3)
        draw.text((x_min, y_min - 20), str(detection["number"]), fill="red")
    
    return img

annotated_image = draw_detections(image_url, detections.boxes)
display(annotated_image)


## Texto + Imagem

Signatures podem combinar campos de imagem e texto. VQA (Visual Question Answering) é trivial.


In [ ]:
class VisualQuestionAnswering(dspy.Signature):
    """Responde perguntas sobre o conteúdo de uma imagem."""
    image: dspy.Image = dspy.InputField(desc="Imagem para análise")
    question: str = dspy.InputField(desc="Pergunta sobre a imagem")
    answer: str = dspy.OutputField(desc="Resposta detalhada baseada na imagem")

vqa = dspy.Predict(VisualQuestionAnswering)
result = vqa(
    image=numbers_image,
    question="Quantos números diferentes aparecem na imagem? Qual é a soma deles?"
)
result.answer


## Módulos Avançados

`ChainOfThought`, `ReAct` e outros módulos funcionam com imagens sem modificação.


In [ ]:
class ImageAnalysis(dspy.Signature):
    """Análise detalhada de imagem com raciocínio passo a passo."""
    image: dspy.Image = dspy.InputField(desc="Imagem para análise")
    analysis: str = dspy.OutputField(desc="Análise detalhada com raciocínio")
    key_findings: str = dspy.OutputField(desc="Principais descobertas")

image_analyzer = dspy.ChainOfThought(ImageAnalysis)
result = image_analyzer(image=numbers_image)
result


## Módulos Customizados

Combine múltiplas operações visuais seguindo o mesmo padrão de módulos textuais.


In [ ]:
class ComprehensiveImageAnalysis(dspy.Module):
    """Módulo que combina detecção e análise de imagem."""
    
    def __init__(self):
        self.detector = dspy.Predict(NumberDetections)
        self.analyzer = dspy.ChainOfThought(ImageAnalysis)
    
    def forward(self, image: dspy.Image):
        detections = self.detector(image=image)
        analysis = self.analyzer(image=image)
        
        return dspy.Prediction(
            detections=detections.boxes,
            analysis=analysis.analysis,
            findings=analysis.key_findings
        )

comprehensive_analyzer = ComprehensiveImageAnalysis()
result = comprehensive_analyzer(image=numbers_image)
result


## Custo e Performance

VLMs são mais caros que LLMs. DSPy fornece histórico de custos por chamada via `lm.history[-1]["cost"]`.


In [ ]:
if vision_model.history:
    last_call = vision_model.history[-1]
    cost = last_call.get("cost", 0)
    tokens = last_call.get("tokens", {})
    
    print(f"Custo: ${cost:.6f}")
    print(f"Tokens: {tokens}")


## Context Switching

Use `dspy.context()` para alternar modelos VLM por tarefa.


In [ ]:
specialized_vlm = dspy.LM("gemini/gemini-2.5-flash", api_key=GEMINI_API_KEY)

with dspy.context(lm=specialized_vlm):
    result = image_describer(image=numbers_image)
    result.description


## Conclusões

**Insights:**
- `dspy.Image` abstrai manipulação de imagens
- Mesma interface para URLs, arquivos, bytes, PIL
- Outputs estruturados funcionam automaticamente
- Módulos avançados funcionam sem modificação
- Context switching para modelos diferentes

**Uso ideal:** OCR, análise de documentos, detecção de objetos, VQA, análise de gráficos.

**Limitações:** Custo maior, latência maior, requer modelos VLM especializados.
